In [ ]:
import IPython.display
import sys
import json
from rdflib import *
import glob
import os
import re
from SPARQLWrapper import SPARQLWrapper, JSON, BASIC

In [ ]:
def loadAuthFile():
    with open('auth_add_places.txt') as f:
        auth =  f.readlines()
        auth = [x.replace('\n','') for x in auth]
    return auth

In [ ]:
def getSPARQLResults(queryString, auth):
    result = None
    sparql = SPARQLWrapper(auth[0] + "KGRC4SI_20221208_r2")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials(auth[1], auth[2])
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    try :
        json = sparql.query().convert()
        bindings = json['results']['bindings']
        result = bindings
    except  Exception as e:
        print(e)
    return result

In [ ]:
def getAskSPARQLResults(queryString, auth):
    result = None
    sparql = SPARQLWrapper(auth[0] + "KGRC4SI_20221208_r2")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials(auth[1], auth[2])
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    try :
        json = sparql.query().convert()
        result = json['boolean']
    except  Exception as e:
        print(e)
    return result

In [ ]:
def checkClass(instance):
    auth = loadAuthFile()
    query = """
            PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
            ASK { ho:%s ?p ?o }
        """
    result = getAskSPARQLResults(query % class_name, auth)
    return result

In [ ]:
def getActivity():
    auth = loadAuthFile()
    query = """
            PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
            PREFIX ex: <http://example.org/virtualhome2kg/instance/>
            PREFIX vh2kg: <http://example.org/virtualhome2kg/ontology/>
            SELECT DISTINCT ?s 
            WHERE { 
                ?s vh2kg:virtualHome ?scene . 
            }
        """
    result = getSPARQLResults(query, auth)
    return result

In [ ]:
base = Namespace("http://example.org/virtualhome2kg/instance/")
onto = Namespace("http://example.org/virtualhome2kg/ontology/")
ho = Namespace("http://www.owl-ontologies.com/VirtualHome.owl#")
g = Graph()
g.bind("example.py", base)
g.bind("vh2kg", onto)
g.bind("ho", ho)
g.bind("owl", OWL)
results = getActivity()
rules = {
    "clean": "HouseCleaning",
    "television": "Leisure",
    "eat": "EatingDrinking",
    "drink": "EatingDrinking",
    "relax": "Leisure",
    "watch": "Leisure",
    "research_on_computer": "Work",#Added on 2023/07/10
    "put": "HouseArrangement",
    "cook": "FoodPreparation",#Added on 2023/07/07
    "homework": "Work",#Added on 2023/07/07
    "fall": "Abnormal",#Added on 2023/07/07
    "fold": "HouseArrangement",#Added on 2023/07/07
    "out_of_bed": "BedTimeSleep",#Added on 2023/07/07
    "beverage": "EatingDrinking",#Added on 2023/07/07
    "legopp": "PhysicalActivity",#Added on 2023/07/10
    "cereal": "FoodPreparation",#Added on 2023/07/10
    "pick_up_dirty": "HouseArrangement",#Added on 2023/07/10
    "place_": "HouseArrangement",#Added on 2023/07/10
    "brushing_teeth": "HygieneStyling",#Added on 2023/07/10
    "bedtime": "BedTimeSleep",#Added on 2023/07/10
    "read_textbook": "Work",#Added on 2023/07/10
    "rinse_toothbrush": "HygieneStyling",#Added on 2023/07/10
    "disorientation": "Abnormal",#Added on 2023/07/10
    "stretch": "PhysicalActivity",#Added on 2023/07/10
    "social_media": "Leisure",#Added on 2023/07/10
    "squat": "PhysicalActivity",#Added on 2023/07/10
    "stand_on": "Abnormal",#Added on 2023/07/10
    "straddle": "PhysicalActivity",#Added on 2023/07/10
    "trash": "HouseArrangement",#Added on 2023/07/10
    "use_phone": "Leisure",#Added on 2023/07/10
    "memory_loss": "Abnormal",#Added on 2023/07/10
    "write_notes": "Work",#Added on 2023/07/10
}
for result in results:
    activity = result["s"]["value"]
    activity_name = activity.replace("http://example.org/virtualhome2kg/instance/", "")
    class_name = re.sub("[0-9]*_scene[0-7]", "", activity_name)
    if checkClass(activity) == False:
        g.add((ho[class_name], RDF.type, OWL.Class))
        parent = [rules[x] for x in rules if x in class_name][0]
        g.add((ho[class_name], RDFS.subClassOf, ho[parent]))

In [ ]:
output_path="../dataset/Dataset_12082022_revised2/rdf20230320/add_classes.ttl"
g.serialize(destination=output_path, format="ttl")

In [ ]:
g.parse("../ontology/vh2kg_schema.ttl", format="ttl")

In [ ]:
output_path="../dataset/Dataset_12082022_revised2/rdf20230320/vh2kg_schema_2023-07.ttl"
g.serialize(destination=output_path, format="ttl")